# Data Conversion of SeaFET .hex data
This python notebook shows how to convert a .hex file into pandas dataframe with scientific values.

Please contact [SBS customer support](https://www.seabird.com/support) for help or to request additional features.

## Init

The initialization code section below is used to import required libraries.

In [1]:
# Native imports
import os

# Third-party imports
import numpy as np
import pandas as pd

# Sea-Bird imports
import seabirdscientific.conversion as conv
import seabirdscientific.instrument_data as id
import seabirdscientific.visualization as viz
import example_data.example_coefficients as ec

## [Data Conversion](#proc-list)

This section shows how to convert raw data contained in a .hex file into scientific units for the instruments that follow:  
- SeaFET

In [2]:
hex_file = os.path.join("example_data", "SeaFET_V2", "SeaFET_V2.hex")

# Convert raw hexadecimal string to raw frequencies
raw_data = id.read_hex_file(
    filepath=hex_file,
    instrument_type=id.InstrumentType.SeaFET2,
    is_shallow=True
)

In [3]:

raw_data

,vrs external,vrs internal,ph temperature,vk,ib,ik,relative humidity,internal temperature,date time,error flag
0,5106181.0,5096276.0,1002679.0,5960288.0,8381912.0,8384541.0,17632.0,26016.0,2025-09-15 21:35:48,0.0


In [6]:
# Convert raw frequencies to scientific values

# Salinity and pressure data used in external pH calculation
# If you have salinity and pressure data from another sensor, input it here. Otherwise adjust these reference values.
salinity = np.full(len(raw_data), 30) # PSU
pressure = np.full(len(raw_data), 0) # dbar

temperature = conv.convert_seafet_temperature(
    raw_temp=raw_data["ph temperature"].values, 
    coefs=ec.temeprature_seafet_coefs
)

ph_internal = conv.convert_internal_seafet_ph(
    ph_counts=raw_data["vrs internal"].values,
    temperature=temperature,
    coefs=ec.ph_internal_coefs
)

ph_external = conv.convert_external_seafet_ph(
    ph_counts=raw_data["vrs external"].values,
    temperature=temperature,
    salinity=salinity,
    pressure=pressure,
    coefs=ec.ph_external_coefs

)

# Flag to be used in data processing
flag = np.zeros(len(raw_data))

data = pd.DataFrame({
    "ph_internal":ph_internal,
    "ph_external":ph_external,
    "date_time":raw_data["date time"],
    "flag":flag
})

data

,ph_internal,ph_external,date_time,flag
0,8.348504,7.544718,2025-09-15 21:35:48,0.0


## [Data Plotting](#proc-list)

In [ ]:
config = viz.ChartConfig(
    title="SeaFET Data Conversion",
    x_names=["date_time"],
    y_names=["ph_internal", "ph_external"],
    z_names=[],
    chart_type="overlay",
    plot_loop_edit_flags=False,
    lift_pen_over_bad_data=True,
)

chart_data = viz.ChartData(data, config)
fig = viz.plot_xy_chart(chart_data, config)

# plotly customizations
fig["layout"]["yaxis"]["autorange"] = "reversed"
fig.data[0].name = "pH Internal"
fig.data[1].name = "pH External"
fig["layout"]["yaxis"]["title"] = "pH Internal"
fig["layout"]["yaxis2"]["title"] = "pH External"


fig.update_layout(height=800)
fig.show()